In [ ]:
import torch
import collections
import math
from torch import nn


In [ ]:
class Encoder(nn.Module):
    def __init__(self, vocab_size,embed_size,hidden_size,num_layer,dropout=0,*args, **kwargs):
        super().__init__(*args, **kwargs)
        self.embedding = nn.Embedding(vocab_size,embed_size)
        self.rnn = nn.GRU(embed_size,hidden_size=hidden_size,num_layers=num_layer,dropout=dropout)
    def foward(self,X,*args):
        X = self.embedding(X)
        X = X.permute(1,0,2)
        outputs , state = self.rnn(X)

        return outputs,state
class Decoder(nn.Module):
    
    def __init__(self,vocab_size,embed_size,num_hiddens,num_layer,drouptout=0,*args, **kwargs):
        super().__init__(*args, **kwargs)

        self.embedding = nn.Embedding(vocab_size,embed_size)
        self.rnn = nn.GRU(embed_size + num_hiddens,num_hiddens,num_layer,dropout=drouptout)
        self.dense = nn.Linear(num_hiddens,vocab_size)
    def init_state(self,enc_outputs,*agrs):
        return enc_outputs[1]
    def forward(self,X,state):
        X = self.embedding(X).permute(1,0,2)
        context = state[-1].repeat(X.shape[0],1,1)
        X_and_context = torch.cat((X,context),2)
        output,state = self.rnn(X_and_context,state)
        output = self.dense(output).permute(1,0,2)
        return output,state
